In [ ]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, TextStreamer
from datetime import datetime
import re

## params

In [ ]:
max_seq_length=512
dtype = None
model_name = "dsr18b"  # your model name here
tokenizer = AutoTokenizer.from_pretrained(model_name) # use the same tokenizer
tokenizer.padding_side = "right" # padding side=left(default)
# print("padding_side:", tokenizer.padding_side)

## wandb visualization

In [ ]:
import wandb

wandb.login(key="you key here")
run = wandb.init(
    project='LoRA SFT in ingratiation dataset',
    job_type="training",
    anonymous="allow"
)

## get response by one request

In [ ]:
streamer = TextStreamer(tokenizer) # streamer ouput
# fix commas in json string
def fix_json_commas(json_string):
    json_string = re.sub(r'([^,{])(\"课程名称\")', r'\1,\2', json_string)
    json_string = re.sub(r'([^,{])(\"分类\")', r'\1,\2', json_string)
    json_string = re.sub(r',\s*}', '}', json_string)
    return json_string

@torch.no_grad() # shut down the gradient calculation
def get_response(input_message):
    inputs = tokenizer(input_message, return_tensors="pt", padding=True).to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.3,
        repetition_penalty=1.2,
        use_cache=True,
        do_sample=False,
        streamer=streamer,  # streamer 
    )
    print(f"outputs={outputs}") # check model outputs

    # get the generated response
    response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response_text.split("</think>")[-1].strip()
    
    # clean the response
    cleaned_res = response_text.replace(input_message, '').strip().replace('```json', '').replace('```', '').replace(' ', '').replace('\n', '').replace('\\"', '"')
    cleaned_res = re.sub("'", '"', cleaned_res)
    cleaned_res = re.sub(r':""(.*?)""', r':"\1"', cleaned_res)
    cleaned_res = fix_json_commas(cleaned_res)

    answer = response_text.split("</think>")[-1].strip()
    return answer

## get test for prompt

In [ ]:
prompt_style = """你是教育学专家，正在对学习者在线课堂中的弹幕进行行为分类。
[分类规则]={{
    "观点遵从"：明确或间接地附和教师观点，表达对教师论点或行为的一致认同。
    "恭维他人"：直接或间接赞美教师、强调教师的重要性与优点。
    "自我展现"：突出自己的见解、经验或成就，以期引起教师注意。
    "施恩他人"：表达对教师或同学的帮助意愿，用行动或建议来支持他人。
}}
### 指令:
请只针对本次问题生成输出，课程名称包含场景信息，回答仅输出JSON格式，不在4类分类中请输出"无"
思考后请严格按照 JSON 输出，禁止输出任何额外文本、注释、思考过程等。只能是 JSON！

[输出格式]=
{{
    "分类": str分类
}}
### 问题:
{}
"""

In [ ]:
question = "{'弹幕内容': '太形象了', '课程名称': '数据结构'}属于什么分类？"

In [ ]:
res = get_response(prompt_style.format(question))

## get response in batch

In [ ]:
def extract_json_from_text(text):
    """
    to get structure of json in {"分类": "XXX"} 
    """
    json_pattern = r'(\{.*?"分类".*?\})'
    matches = re.findall(json_pattern, text, re.DOTALL)
    
    for match in matches:
        try:
            parsed_json = json.loads(match)  # from string to json
            return parsed_json
        except json.JSONDecodeError:
            continue  
    return None

@torch.no_grad()
def get_response_batch(questions):
    inputs = tokenizer(questions, return_tensors="pt", padding=True, truncation=True, max_length=1024).to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.3,
        repetition_penalty=1.2,
        use_cache=True,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
    )
    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    responses = []
    for question, response_text in zip(questions, decoded_outputs):
        try:
            response_text = response_text.split("</think>")[-1].strip()
            cleaned_res = response_text.replace(question, '').strip().replace('```json', '').replace('```', '').replace(' ', '').replace('\n', '').replace('\\"', '"')
            cleaned_res = re.sub("'", '"', cleaned_res)
            cleaned_res = re.sub(r':""(.*?)""', r':"\1"', cleaned_res)
            cleaned_res = fix_json_commas(cleaned_res)
            parsed = json.loads(cleaned_res)
            if isinstance(parsed, list):
                parsed = parsed[0] if parsed else {}
            responses.append(parsed)
        except json.JSONDecodeError:
            # **try get json directly**
            extracted_json = extract_json_from_text(cleaned_res)
            if extracted_json:
                parsed = extracted_json 
                responses.append(parsed)
            else:
                eee = {"error": "cannot find effctive json"}
                responses.append({"error": str(eee)})
        except Exception as e:
            responses.append({"error": str(e)})
    return responses


In [ ]:
# load train, valid, test data
with open('Ingratiation/train_data.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
with open('Ingratiation/valid_data.json', 'r', encoding='utf-8') as f:
    valid_data = json.load(f)
with open('Ingratiation/test_data.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)

In [ ]:
def process_data(data, data_type="train", BATCH_SIZE=8):
    """
    get responese in batch
    :param data: data list with question and response
    :param data_type: train, valid, test
    :param BATCH_SIZE: batch size
    """
    
    results = []
    error_logs = []
    
    # Assemble questions in batches and record the results
    for i in range(0, len(data), BATCH_SIZE):
        # to check the progress
        if i % 100 == 0:
            print(f"data {i} is processing...")
        batch_data = data[i : i + BATCH_SIZE]
        questions_batch = [prompt_style.format(item["Question"]) for item in batch_data]

        # ger responses in batch
        batch_responses = get_response_batch(questions_batch)
        # print(f"data {i}：{batch_responses}")
        for j, response_data in enumerate(batch_responses):
            original_response = batch_data[j]["Response"]
            original_response_gt = original_response["分类"]
            # error handling
            if (
                "error" in response_data 
                or not all(key in response_data for key in ["分类"])
            ):
                error_logs.append({
                    "Question": batch_data[j]["Question"],
                    "原分类": original_response_gt,
                    "错误信息": response_data.get("error", "响应缺少必要字段")
                })
                continue

            # results append
            results.append({
                "Question": batch_data[j]["Question"],
                "分类": response_data["分类"],
                "原分类": original_response_gt
            })

    # file output
    if data_type.lower() == "train":
        result_file = f"df_train_{current_time}.xlsx"
        error_file = f"error_train_log_{current_time}.xlsx"
    elif data_type.lower() == "after_train":
        result_file = f"df_after_train_{current_time}.xlsx"
        error_file = f"error_after_train_log_{current_time}.xlsx"
    elif data_type.lower() == "after_test":
        result_file = f"df_after_test_{current_time}.xlsx"
        error_file = f"error_after_test_log_{current_time}.xlsx"
    else:
        result_file = f"df_test_{current_time}.xlsx"
        error_file = f"error_test_log_{current_time}.xlsx"
    df_result = pd.DataFrame()
    df_error = pd.DataFrame()

    # load df
    if results:
        df_result = pd.DataFrame(results)
        df_result.to_excel(result_file, index=False)
        print(f"{data_type} 处理完成，结果已保存至 {result_file}")
    
    if error_logs:
        df_error = pd.DataFrame(error_logs)
        df_error.to_excel(error_file, index=False)
        print(f"{data_type} 处理完成，错误日志已保存至 {error_file}")

    return df_result,df_error

## LoRA-Finetuning

In [ ]:
train_prompt_style = """你是教育学专家，正在对学习者在线课堂中的弹幕进行行为分类。
[分类规则]={{
    "观点遵从"：明确或间接地附和教师观点，表达对教师论点或行为的一致认同。
    "恭维他人"：直接或间接赞美教师、强调教师的重要性与优点。
    "自我展现"：突出自己的见解、经验或成就，以期引起教师注意。
    "施恩他人"：表达对教师或同学的帮助意愿，用行动或建议来支持他人。
}}
### 指令:
请只针对本次问题生成输出，课程名称包含场景信息，回答仅输出JSON格式，不在4类分类中请输出"无"
思考后请严格按照 JSON 输出，禁止输出任何额外文本、注释、思考过程等，最终答案只能是 JSON！

[输出格式]=
{{
    "分类": str分类
}}
### 问题:
{}
### 回答:
<think>
{}
</think>
### 最终答案：
{}
"""

mask question part in labels and set cot and output weight 

In [ ]:
# get weights 
w_input = 0.0
w_cot = 1.0
w_output = 5.0
token_length = 512

def formatting_prompts_func(examples):
    # get think and output part
    COT_PATTERN = re.compile(r'<think>\n.*?\n</think>', re.DOTALL)
    OUTPUT_MARKER = re.compile(r'### 最终答案：\n')
    # get eos token
    EOS_TOKEN = tokenizer.eos_token_id if "tokenizer" in globals() else "<EOS>"

    if isinstance(EOS_TOKEN, str):
        EOS_TOKEN = tokenizer.convert_tokens_to_ids(EOS_TOKEN)

    # get question, cot and response
    input_texts = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    
    processed = {
        "input_ids": [],
        "attention_mask": [],
        "labels": [],
        "weight_mask": [],
        "text": [],
        "cot_positions": [],  # cot tokens index [start, end]
        "output_positions": []  # output tokens index [start, end]
    }

    for i in range(len(input_texts)):
        input_text = str(input_texts[i]).strip()
        cot = str(cots[i]).strip()
        output = str(outputs[i]).strip()
        
        full_text = train_prompt_style.format(input_text, cot, output)
        processed["text"].append(full_text)  

        # get cot position
        cot_match = COT_PATTERN.search(full_text)
        cot_start, cot_end = (cot_match.start(), cot_match.end()) if cot_match else (0, 0)

        # get output position
        output_match = OUTPUT_MARKER.search(full_text)
        out_start = output_match.end() if output_match else len(full_text)
        out_end = len(full_text)
        
        # Tokenization
        encoded = tokenizer(
            full_text,
            return_offsets_mapping=True,  #
            truncation=True,
            max_length=token_length-1,
            add_special_tokens=False
        )
        
        # transform char position to token position
        def char_pos_to_token_pos(char_pos):
            for token_idx, (start, end) in enumerate(encoded["offset_mapping"]):
                if start <= char_pos < end:  
                    return token_idx
            return -1  # not found

        # get cot and output tokens index
        cot_start_token = char_pos_to_token_pos(cot_start)
        cot_end_token = char_pos_to_token_pos(cot_end - 1) 
        out_start_token = char_pos_to_token_pos(out_start)
        out_end_token = char_pos_to_token_pos(out_end - 1)

        processed["cot_positions"].append([cot_start_token, cot_end_token])
        processed["output_positions"].append([out_start_token, out_end_token])
        
        # initialize labels and weight_mask
        weight_mask = [w_input] * len(encoded["input_ids"])
        labels = [-100] * len(encoded["input_ids"])

        # set cot and output weight
        for idx, (start, end) in enumerate(encoded["offset_mapping"]):
            if start >= len(full_text):
                continue
            # <think> and </think> weight setting
            is_start_tag = (cot_start <= start < cot_start + 8)  # "<think>\n" in 8 characters
            is_end_tag = (cot_end - 9 <= start < cot_end)       # "\n</think>" in 9 characters
            # cot weight setting
            if start < cot_end and end > cot_start:
                if is_start_tag or is_end_tag:
                    weight_mask[idx] = w_output  
                else:
                    weight_mask[idx] = w_cot     
                labels[idx] = encoded["input_ids"][idx]

            # output weight setting
            if start < out_end and end > out_start:
                labels[idx] = encoded["input_ids"][idx]
                weight_mask[idx] = w_output

        # add EOS token
        input_ids = encoded["input_ids"]
        attention_mask = encoded["attention_mask"]
        
        if len(input_ids) < max_seq_length:
            input_ids.append(EOS_TOKEN)
            attention_mask.append(1)
            labels.append(EOS_TOKEN)  # calculate EOS的loss
            weight_mask.append(w_output)  # get weight of EOS

            # extend index
            processed["output_positions"][-1][1] += 1 

        # Padding
        seq_len = len(input_ids)
        pad_len = token_length - seq_len
        
        processed["input_ids"].append(input_ids + [tokenizer.pad_token_id] * pad_len)
        processed["attention_mask"].append(attention_mask + [0] * pad_len)
        processed["labels"].append(labels + [-100] * pad_len)
        processed["weight_mask"].append(weight_mask + [0.0] * pad_len)

    return processed

## load train and valid data

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("json", data_files="Ingratiation/train_data.json", split="train")
valid_dataset = load_dataset("json", data_files="Ingratiation/valid_data.json", split="train")

train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
eval_dataset = valid_dataset.map(formatting_prompts_func, batched=True)

## rewrite loss function 

In [ ]:
# only for cot and output with weighted average loss
from torch.nn import CrossEntropyLoss
from trl import SFTTrainer
from transformers import TrainingArguments
class CustomSFTTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")  # [batch, seq_len]
        weight_mask = inputs.get("weight_mask")  # [batch, seq_len]
        outputs = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            labels=None, # not use model default loss
        )
        logits = outputs.logits  # [batch, seq_len, vocab_size]
        
        # shift logits, labels 和 weight_mask [1,2,3,4] -> [1,2,3] & [2,3,4]
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        shift_weight = weight_mask[..., 1:].contiguous()
        
        # calulate loss
        loss_fct = CrossEntropyLoss(reduction="none")
        loss_per_token = loss_fct(
            shift_logits.view(-1, shift_logits.size(-1)),
            shift_labels.view(-1)
        ).view(shift_labels.size())

        # get cot and output mask
        cot_mask = (torch.abs(shift_weight - 1.0) < 1e-6) & (shift_labels != -100)
        out_mask = (torch.abs(shift_weight - 5.0) < 1e-6) & (shift_labels != -100)

        # calculate count
        cot_count = max(cot_mask.sum().float(), 1e-6)
        out_count = max(out_mask.sum().float(), 1e-6)

        # loss in different weights
        alpha = 12  # coffiecient of output loss
        
        if cot_mask.any():
            cot_loss = loss_per_token[cot_mask].sum()
        else:
            cot_loss = torch.tensor(0.0, device=loss_per_token.device)

        if out_mask.any():
            out_loss = loss_per_token[out_mask].sum() * alpha
        else:
            out_loss = torch.tensor(0.0, device=loss_per_token.device)

        # weighted average loss
        total_loss = (cot_loss + out_loss) / (cot_count + alpha * out_count)

        del outputs, logits, shift_logits, shift_labels, loss_per_token
        return (total_loss, None) if return_outputs else total_loss
    
    # valid dataset also use the same loss function
    def prediction_step(self, model, inputs, prediction_loss_only=True, ignore_keys=None):
        with torch.no_grad():  # important
            loss = self.compute_loss(model, inputs, return_outputs=False)
        return (loss, None, None)

## load the model

In [ ]:
model, _ = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    # attn_implementation="flash_attention_2",
)
FastLanguageModel.for_training(model)

In [ ]:
# set up training arguments
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth", 
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    warmup_steps=100,
    max_steps=375,
    learning_rate=3e-5,
    optim="adamw_hf",
    weight_decay=0.05,
    lr_scheduler_type="cosine",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    tf32=True,
    gradient_checkpointing=True,
    logging_steps=125,
    evaluation_strategy="steps",
    eval_steps=125,
    save_strategy="steps",
    save_steps=125,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    seed=42,
    output_dir="outputs_ingratiation",
)

In [ ]:
def custom_data_collator(features):
    return {
        "input_ids": torch.tensor([f["input_ids"] for f in features], dtype=torch.long),
        "attention_mask": torch.tensor([f["attention_mask"] for f in features], dtype=torch.long),
        "labels": torch.tensor([f["labels"] for f in features], dtype=torch.long),
        "weight_mask": torch.tensor([f["weight_mask"] for f in features], dtype=torch.float),
    }

trainer = CustomSFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=custom_data_collator,
    remove_unused_columns=False,
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=training_args
)

In [ ]:
import os
os.environ["UNSLOTH_RETURN_LOGITS"] = "1"
# unsloth returns full logits tensors when propagating forward, which typically consumes a lot of video memory.

In [ ]:
# begin training
trainer_stats = trainer.train()

In [ ]:
# get the training stats
wandb.finish()

In [ ]:
# Save the fine-tuned model
new_model_local = "DeepSeek-R1-Ingratiation-COT-0317"
model.save_pretrained(new_model_local) # Local saving
tokenizer.save_pretrained(new_model_local)

## use the lora-finetuned model

In [ ]:
model_name = "DeepSeek-R1-Ingratiation-COT-0317"  
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"

model, _ = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype = dtype,
    # attn_implementation="flash_attention_2",
)
FastLanguageModel.for_inference(model)

In [ ]:
# get train data answer
current_time1 = datetime.now()
df_train_after_SFT_result,df_train_after_SFT_error = process_data(train_data, data_type="after_train", BATCH_SIZE=1)
current_time2 = datetime.now()
print(f'用时{(current_time2-current_time1).total_seconds()}')

In [ ]:
print(len(df_train_after_SFT_result))
print(len(df_train_after_SFT_error))

In [ ]:
# get valid data answer
current_time1 = datetime.now()
df_valid_after_SFT_result,df_valid_after_SFT_error = process_data(valid_data, data_type="after_train", BATCH_SIZE=1)
current_time2 = datetime.now()
print(f'用时{(current_time2-current_time1).total_seconds()}')

In [ ]:
print(len(df_valid_after_SFT_result))
print(len(df_valid_after_SFT_error))

In [ ]:
# get test data answer
current_time1 = datetime.now()
df_test_after_SFT_result,df_test_after_SFT_error = process_data(test_data, data_type="after_test", BATCH_SIZE=1)
current_time2 = datetime.now()
print(f'用时{(current_time2-current_time1).total_seconds()}')

In [ ]:
print(len(df_test_after_SFT_result))
print(len(df_test_after_SFT_error))

## accuracy

In [ ]:
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score
)

In [ ]:
def evaluate_model(true_labels, pred_labels):
    metrics = {
        'Accuracy': accuracy_score(true_labels, pred_labels),
        'Precision(macro)': precision_score(true_labels, pred_labels, average='macro'),
        'Recall(macro)': recall_score(true_labels, pred_labels, average='macro'),
        'F1(macro)': f1_score(true_labels, pred_labels, average='macro'),
        'Cohen Kappa': cohen_kappa_score(true_labels, pred_labels)
    }
    return {k: round(v, 4) for k, v in metrics.items()}

file_map = {
    'before_SFT_train': 'df_train_202503201253.xlsx',
    'before_SFT_test': 'df_test_202503201323.xlsx',
    'after_SFT_train': 'df_after_train_202503222116.xlsx',
    'after_SFT_test': 'df_after_test_202503222130.xlsx'
}
results = {}

In [ ]:
for desc, filename in file_map.items():
    df = pd.read_excel(filename, engine='openpyxl')
    if '原分类' not in df.columns or '分类' not in df.columns:
        raise ValueError(f"{filename} lost '原分类' 或 '分类'")
    metrics = evaluate_model(df['原分类'], df['分类'])
    results[desc] = metrics

results_df = pd.DataFrame(results).T
print(results_df)

In [ ]:
print("\ndiffernce:")

train_diff = results_df.loc['before_SFT_test'] - results_df.loc['before_SFT_train']
train_diff.name = 'train diff'
test_diff = results_df.loc['after_SFT_test'] - results_df.loc['after_SFT_train']
test_diff.name = 'test diff'

comparison = pd.concat([train_diff, test_diff], axis=1).T
print(comparison)